# Практична робота

## 1. Проглянути дані

In [4]:
from conllu import parse
from collections import OrderedDict

PATH = 'UD_Ukrainian-IU'

def load_treebank(file):
    with open(PATH + '/' + file) as f:
        data = f.read()

    return parse(data)
    
train_trees = load_treebank('uk_iu-ud-train.conllu')
test_trees = load_treebank('uk_iu-ud-dev.conllu')

In [2]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

def print_tree(tree):
    for node in tree:
        head = node['head']
        print('{} <-- {}'.format(node['form'], tree[head - 1]['form'] if head > 0 else 'root'))

def golden_rels(tree):
    relations = []
    for node in tree:
        head = node['head']
        relations.append((tree[head - 1]['id'] if head > 0 else 0, node['id']))
    return relations

tree = train_trees[0]
golden_rels(tree)

[(2, 1),
 (6, 2),
 (4, 3),
 (2, 4),
 (4, 5),
 (0, 6),
 (8, 7),
 (6, 8),
 (10, 9),
 (8, 10),
 (10, 11),
 (13, 12),
 (11, 13),
 (6, 14)]

In [3]:
tree[0]

OrderedDict([('id', 1),
             ('form', 'У'),
             ('lemma', 'у'),
             ('upostag', 'ADP'),
             ('xpostag', 'Spsl'),
             ('feats', OrderedDict([('Case', 'Loc')])),
             ('head', 2),
             ('deprel', 'case'),
             ('deps', [('case', 2)]),
             ('misc',
              OrderedDict([('Id', '0003'),
                           ('LTranslit', 'u'),
                           ('Translit', 'U')]))])

## 2. Побудувати статичного оракула, який визначає послідовність дій для побудови дерева

#### Визначення дій:
- якщо черга порожня, REDUCE
- якщо є залежність від вершини стеку до вершини черги, RIGHT
- якщо є залежність від вершини черги до вершини стеку, LEFT
- якщо вершина стеку вже має батька:
    - якщо вершина черги має інші залежності серед елементів стеку, REDUCE
    - в іншому випадку, SHIFT
- в іншому випадку, SHIFT

In [33]:
SHIFT = "shift"
REDUCE = "reduce"
RIGHT = "right"
LEFT = "left"

def is_left_arc(top_stack, top_queue):
    return top_stack['head'] == top_queue['id']

def it_right_arc(top_stack, top_queue):
    return top_queue['head'] == top_stack['id']

def has_parent(item, relations):
    return item['id'] in [r[0] for r in relations]

def has_head_in_stack(top_queue, stack):
    return (top_queue['head'] < stack[-1]['id'] or \
            next((s for s in stack if s['head'] == top_queue['id']), None))

def oracle(stack, queue, relations):
    top_stack = stack[-1]
    top_queue = queue[0] if queue else None
    
    if not top_queue:
        return REDUCE
    elif is_left_arc(top_stack, top_queue):
        return LEFT
    elif it_right_arc(top_stack, top_queue):
        return RIGHT
    elif has_parent(top_stack, relations) and \
         has_head_in_stack(top_queue, stack):
        return REDUCE
    else:
        return SHIFT

#### Застосування дій на дереві:
- SHIFT - перенести вершину черги у стек
- RIGHT - проставити залежність від вершини стеку до вершини черги і перенести вершину черги у стек
- LEFT - проставити залежність від вершини черги до вершини стеку і видалити вершину стеку
- REDUCE - видалити вершину стеку

In [33]:
def apply(action, stack, queue, relations):
    if action == SHIFT:
        stack.append(queue.pop(0))
    elif action == RIGHT:
        relations.append((queue[0]['id'], stack[-1]['id']))
        stack.append(queue.pop(0))
    elif action == LEFT:
        relations.append((stack[-1]['id'], queue[0]['id']))
        stack.pop()
    elif action == REDUCE:
        stack.pop()
    else:
        raise 'unknown action'

def traverse(tree):
    stack, queue, relations = [ROOT], tree[:], []

    while queue or stack:
        action = oracle(stack, queue, relations)
            
        yield(stack, queue, relations, action)

        apply(action, stack, queue, relations)

In [12]:
for _, _, _, action in traverse(tree):
    print(action)

shift
left
shift
shift
left
right
right
reduce
reduce
left
right
shift
left
right
shift
left
right
right
shift
left
right
reduce
reduce
reduce
reduce
right
reduce
reduce
reduce


## 3. Виділити ознаки

Написати функцію, яка дістає з дерева набір переходів та набір ознак для цих переходів.

In [13]:
def extract_features(stack, queue, *args):
    features = {}
    
    if stack:
        features['stk-0-form'] = stack[-1]['form']
        features['stk-0-lemma'] = stack[-1]['lemma']
        features['stk-0-tag'] = stack[-1]['upostag'] 
    if len(stack) > 1:
        features['stk-1-tag'] = stack[-2]['upostag']
        
    if queue:
        features['que-0-form'] = queue[0]['form']
        features['que-0-lemma']= queue[0]['lemma']
        features['que-0-tag']= queue[0]['upostag']
    if len(queue) > 1:
        features['que-1-form'] = queue[1]['form']
        features['que-1-tag'] = queue[1]['upostag']
    if len(queue) > 2:
        features['que-2-tag'] = queue[2]['upostag']
    if len(queue) > 3:
        features['que-3-tag'] = queue[3]['upostag']
    
    return features


def get_data(tree):
    features, labels = [], []

    for stack, queue, relations, action in traverse(tree):
        features.append(extract_features(stack, queue, relations, tree))
        labels.append(action)

    return features, labels


features, labels = get_data(tree)
print(features[0])
print(labels[0])

{'stk-0-form': 'ROOT', 'stk-0-lemma': 'ROOT', 'stk-0-tag': 'ROOT', 'que-0-form': 'У', 'que-0-lemma': 'у', 'que-0-tag': 'ADP', 'que-1-form': 'домі', 'que-1-tag': 'NOUN', 'que-2-tag': 'ADJ', 'que-3-tag': 'NOUN'}
shift


## 4. Дістати тренувальні та тестувальні дані

- Пройтися по всіх деревах у тренувальній вибірці та дістати всі переходи з ознаками.
- Пройтися по всіх деревах у тестувальній вибірці та дістати всі переходи з ознаками.
- Тестувальні дані беремо з “uk_iu-ud-dev.conllu”.

In [14]:
def prepare_data(trees):
    features, labels = [], []
    for tree in trees:
        try:
            f, l = get_data([t for t in tree if type(t["id"])==int])
            features += f
            labels += l  
        except Exception as e:
            # print('Error occured!')
            raise(e)
            pass
    return features, labels

In [16]:
train_features, train_labels = prepare_data(train_trees)
print(len(train_features), len(train_labels))

190298 190298


In [37]:
test_features, test_labels = prepare_data(test_trees)  
print(len(test_features), len(test_labels))

25820 25820


## 5. Натренувати класифікатор
- Векторизувати ознаки тренувальної/тестувальної вибірки.
- Натренувати класифікатор на тренувальній вибірці.
- Протестувати класифікатор на тестувальній вибірці.

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import confusion_matrix, classification_report

vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)

print("Total number of features: ", len(vec.get_feature_names()))

Total number of features:  111126


In [19]:
X_train = vec.transform(train_features)
y_train  = train_labels

X_test = vec.transform(test_features)
y_test = test_labels

In [20]:
lrc = LogisticRegression(multi_class="multinomial", verbose=1)
lrc.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/h8si2w8/.pyenv/versions/3.8.2/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.8s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)

In [21]:
y_pred = lrc.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print()
print(classification_report(y_test, y_pred))

[[5498  241  290  342]
 [ 400 5345  875  255]
 [ 210  576 4734  476]
 [ 311  118  383 5766]]

              precision    recall  f1-score   support

        left       0.86      0.86      0.86      6371
      reduce       0.85      0.78      0.81      6875
       right       0.75      0.79      0.77      5996
       shift       0.84      0.88      0.86      6578

    accuracy                           0.83     25820
   macro avg       0.83      0.83      0.83     25820
weighted avg       0.83      0.83      0.83     25820



## 6. Вирахувати unlabeled attachment score (UAS)
- Скільки залежностей у побудованому дереві збіглося з еталонним деревом?
- Порахувати на тестувальній вибірці.

In [38]:
def intersect(s1, s2):
    return len(set(s1).intersection(set(s2)))

def new_shiny_oracle(stack, queue, relations):
    features = extract_features(stack, queue)
    vec_features = vec.transform([features])
    return lrc.predict(vec_features)[0]

def dep_parse(tree):
    stack, queue, relations = [ROOT], tree[:], []

    while queue or stack:
        action = new_shiny_oracle(stack, queue, relations)

        apply(action, stack, queue, relations)

    return sorted(relations)

tree = test_trees[205]
golden = [(node['id'], node['head']) for node in tree]
predicted = dep_parse(tree)
tp = intersect(golden, predicted)
total = len(golden)
round(tp/total, 2)

0.63

In [39]:
def scores(trees):
    total, tp, errors = 0, 0, 0
    for i, tree in enumerate(trees):
        try:
            tree = [t for t in tree if type(t["id"])==int]
            golden = [(node['id'], node['head']) for node in tree]
            predicted = dep_parse(tree)
            total += len(tree)
            tp += intersect(golden, predicted)
        except Exception as e:
            # print(tree)
            # print('Error in tree №': i)
            # print(total, tp, errors, round(tp/total, 2))
            errors += 1
            # raise(e)
            pass
    return total, errors, tp, round(tp/total, 2)

def print_scores(total, errors, tp, uas):
    print("Total:", total)
    print("Errors:", errors)
    print("Correct:", tp)
    print("UAS:", round(tp/total, 2))
    
def uas_report(trees):
    total, errors, tp, uas = scores(test_trees)
    print_scores(total, errors, tp, uas)

In [32]:
scores_report(test_trees)

Total: 12502
Errors: 1
Correct: 8635
UAS: 0.69
